In [ ]:
import pandas as pd

df = pd.read_csv('../results/question_type.csv', index_col=0)

# df['question_type'].loc[df['question_type']=='factoid'] = 1.0
# df['question_type'].loc[df['question_type']=='boolean'] = 2.0
# df['question_type'].loc[df['question_type']=='count'] = 3.0
# df['question_type'].loc[df['question_type']=='list'] = 4.0

# df['question'] = df['question'].apply(lambda x: ' '.join(x.split(' ')[0:4]))
df['question'] = df['question'].apply(lambda x: ' '.join(x.split(' ')))
df[df['question_type']=='list']

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, HashingVectorizer, TfidfVectorizer

vectorizer = CountVectorizer(lowercase=False)
# vectorizer = HashingVectorizer(lowercase=False)
# vectorizer = TfidfVectorizer()

In [ ]:
from sklearn.model_selection import train_test_split

questions = df['question']
# y = df['question_type'].astype('int')
y = df['question_type']

questions_train, questions_test, y_train, y_test = train_test_split( questions, y, test_size=0.25, random_state=1000)

In [ ]:
vectorizer.fit(questions_train)
X_train = vectorizer.transform(questions_train)
X_test  = vectorizer.transform(questions_test)

X_train

In [ ]:
from sklearn.linear_model import LogisticRegression

classifier = LogisticRegression()
classifier.fit(X_train, y_train)
score = classifier.score(X_test, y_test)

print("Accuracy:", score)

In [ ]:
def check_count(question):
    count_list = ['count']
    if 'how many' in question.lower():
        return True
    if question.lower().split(' ')[0] in count_list:
        return True

    return False


def check_boolean(question):
    bool_list = ['was', 'is', 'are', 'does', 'did']
    q_list = question.lower().split(' ')
    return q_list[0] in bool_list


def check_factoid(question):
    word_list = ['what', 'when', 'which', 'who', 'how', 'where', 'whom', 'whose', 'why']
    return (question.split(' ')[0].lower() in word_list) or (question.split(' ')[1].lower() in word_list)


def check_list(question):
    list_list = ['give me', 'name', 'list', 'tell me',  ]
    q_list = question.lower().split(' ')
    q = ' '.join(q_list)
    if q_list[0] in list_list:
        return True
    if ' '.join(q_list[0:2]) in list_list:
        return True
    return False

def check_quest(quest):
    if check_count(quest):
        return 'count'
    elif check_boolean(quest):
        return 'boolean'
    elif check_factoid(quest):
        return 'factoid'
    elif check_list(quest):
        return 'list'
    else:
        return ''

In [ ]:
df['approx_question_type'] = df['question'].apply(lambda x: check_quest(x))

df['equal'] = df.apply(lambda x: 1 if x['question_type'] == x['approx_question_type'] else 0, axis=1)
df['equal'][df['approx_question_type']!=''].mean()
# df[df['approx_question_type']=='']

In [ ]:
df_rest = df[df['approx_question_type']=='']
df_rest

In [ ]:
X_rest = vectorizer.transform(df_rest['question'])

score_rest = classifier.score(X_rest, df_rest['question_type'])

print("Accuracy:", score_rest)

In [ ]:
classifier.predict(X_rest)

In [ ]:
df['approx_question_type'] = df.apply(lambda x: classifier.predict(vectorizer.transform([x['question']]))[0] if x['approx_question_type'] == '' else x['approx_question_type'], axis=1)
df

In [ ]:
df['equal'] = df.apply(lambda x: 1 if x['question_type'] == x['approx_question_type'] else 0, axis=1)
df['equal'][df['approx_question_type']!=''].mean()